In [ ]:
import os

pre_trained_path = '/content/drive/MyDrive/종합설계/myslowfast/pre_trained'
imagenet_path = os.path.join(pre_trained_path,'resnet50_imagenet.h5')
places365_path = os.path.join(pre_trained_path,'resnet50_places365.h5')
#places365_path = os.path.join(pre_trained_path,'resnet50_places365.h5')


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers
from tensorflow.keras.layers import Input,Add,Conv2D, Activation,BatchNormalization,ZeroPadding2D, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model, Sequential
import tensorflow as tf
import tensorflow.keras.backend as K


slow pathway 만들기

In [ ]:

 
 
def conv1_layer(x):    
    x = ZeroPadding2D(padding=(3, 3),name='conv1_pad')(x)
    x = Conv2D(64, (7, 7), strides=(2, 2),name='conv1_conv')(x)
    x = BatchNormalization(name='conv1_bn')(x)
    x = Activation('relu',name='conv1_relu')(x)
    x = ZeroPadding2D(padding=(1,1),name='pool1_pad')(x)
 
    return x   
 
    
 
def conv2_layer(x):         
    x = MaxPooling2D((3, 3), 2, name='pool1_pool')(x)     
 
    shortcut = x
 
    for i in range(1,4):
        if (i == 1):
            x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid',name='conv2_block1_1_conv')(x)
            x = BatchNormalization(name='conv2_block1_1_bn')(x)
            x = Activation('relu',name='conv2_block1_1_relu')(x)
            
            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same',name='conv2_block1_2_conv')(x)
            x = BatchNormalization(name='conv2_block1_2_bn')(x)
            x = Activation('relu',name='conv2_block1_2_relu')(x)
 
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid',name='conv2_block1_3_conv')(x)
            shortcut = Conv2D(256, (1, 1), strides=(1, 1), padding='valid',name='conv2_block1_0_conv')(shortcut)            
            x = BatchNormalization(name='conv2_block1_3_bn')(x)
            shortcut = BatchNormalization(name='conv2_block1_0_bn')(shortcut)
 
            x = Add(name='conv2_block1_add')([x, shortcut])  
            x = Activation('relu',name='conv2_block1_out')(x)
            
            shortcut = x
 
        else:
            x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid',name='conv2_block' + str(i) +'_1_conv')(x)
            x = BatchNormalization(name='conv2_block' + str(i) +'_1_bn')(x)
            x = Activation('relu',name='conv2_block' + str(i) +'_1_relu')(x)
            
            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same',name='conv2_block' + str(i) +'_2_conv')(x)
            x = BatchNormalization(name='conv2_block' + str(i) +'_2_bn')(x)
            x = Activation('relu',name='conv2_block' + str(i) +'_2_relu')(x)
 
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid',name='conv2_block' + str(i) +'_3_conv')(x)
            x = BatchNormalization(name='conv2_block' + str(i) +'_3_bn')(x)
 
            x = Add(name='conv2_block' + str(i) + '_add')([x, shortcut])
            x = Activation('relu',name='conv2_block' + str(i) +'_out')   (x)  
 
            shortcut = x        
    
    return x
 
 
 
def conv3_layer(x):        
    shortcut = x    
    
    for i in range(1,5):     
        if(i == 1):            
            x = Conv2D(128, (1, 1), strides=(2, 2), padding='valid',name='conv3_block1_1_conv')(x)
            x = BatchNormalization(name='conv3_block1_1_bn')(x)
            x = Activation('relu',name='conv3_block1_1_relu')(x)       
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same',name='conv3_block1_2_conv')(x)
            x = BatchNormalization(name='conv3_block1_2_bn')(x)
            x = Activation('relu',name='conv3_block1_2_relu')(x)
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid',name='conv3_block1_3_conv')(x)
            shortcut = Conv2D(512, (1, 1), strides=(2, 2), padding='valid',name='conv3_block1_0_conv')(shortcut)
            x = BatchNormalization(name='conv3_block1_3_bn')(x)
            shortcut = BatchNormalization(name='conv3_block1_0_bn')(shortcut)
 
            x = Add(name='conv3_block1_add')([x, shortcut])  
            x = Activation('relu',name='conv3_block1_out')(x)    
 
            shortcut = x              
        
        else:
            x = Conv2D(128, (1, 1), strides=(1, 1), padding='valid',name='conv3_block' + str(i) + '_1_conv')(x)
            x = BatchNormalization(name='conv3_block' + str(i) + '_1_bn')(x)
            x = Activation('relu',name='conv3_block' + str(i) + '_1_relu')(x)
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same',name='conv3_block' + str(i) + '_2_conv')(x)
            x = BatchNormalization(name='conv3_block' + str(i) + '_2_bn')(x)
            x = Activation('relu',name='conv3_block' + str(i) + '_2_relu')(x)
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid',name='conv3_block' + str(i) + '_3_conv')(x)
            x = BatchNormalization(name='conv3_block' + str(i) + '_3_bn')(x)
 
            x = Add(name='conv3_block' + str(i) + '_add')([x, shortcut])
            x = Activation('relu',name='conv3_block' + str(i) + '_out')(x)
 
            shortcut = x      
            
    return x
 
 
 
def conv4_layer(x):
    shortcut = x        
  
    for i in range(1,7):     
        if(i == 1):            
            x = Conv2D(256, (1, 1), strides=(2, 2), padding='valid',name='conv4_block1_1_conv')(x)
            x = BatchNormalization(name='conv4_block1_1_bn')(x)
            x = Activation('relu',name='conv4_block1_1_relu')(x)
            
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same',name='conv4_block1_2_conv')(x)
            x = BatchNormalization(name='conv4_block1_2_bn')(x)
            x = Activation('relu',name='conv4_block1_2_relu')(x)  
 
            x = Conv2D(1024, (1, 1), strides=(1, 1), padding='valid',name='conv4_block1_3_conv')(x)
            shortcut = Conv2D(1024, (1, 1), strides=(2, 2), padding='valid',name='conv4_block1_0_conv')(shortcut)
            x = BatchNormalization(name='conv4_block1_3_bn')(x)
            shortcut = BatchNormalization(name='conv4_block1_0_bn')(shortcut)
 
            x = Add(name='conv4_block1_add')([x, shortcut])
            x = Activation('relu',name='conv4_block1_out')(x)
 
            shortcut = x               
        
        else:
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid',name='conv4_block' + str(i) + '_1_conv')(x)
            x = BatchNormalization(name='conv4_block' + str(i) + '_1_bn')(x)
            x = Activation('relu',name='conv4_block' + str(i) + '_1_relu')(x)
            
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same',name='conv4_block' + str(i) + '_2_conv')(x)
            x = BatchNormalization(name='conv4_block' + str(i) + '_2_bn')(x)
            x = Activation('relu',name='conv4_block' + str(i) + '_2_relu')(x)
 
            x = Conv2D(1024, (1, 1), strides=(1, 1), padding='valid',name='conv4_block' + str(i) + '_3_conv')(x)
            x = BatchNormalization(name='conv4_block' + str(i) + '_3_bn')(x)            
 
            x = Add(name='conv4_block' + str(i) + '_add')([x, shortcut])
            x = Activation('relu',name='conv4_block' + str(i)  + '_out')(x)
 
            shortcut = x      
 
    return x
 
 
 
def conv5_layer(x):
    shortcut = x    
  
    for i in range(1,4):     
        if(i == 1):            
            x = Conv2D(512, (1, 1), strides=(2, 2), padding='valid',name='conv5_block1_1_conv')(x)
            x = BatchNormalization(name='conv5_block1_1_bn')(x)
            x = Activation('relu',name='conv5_block1_1_relu')(x)
            
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same',name='conv5_block1_2_conv')(x)
            x = BatchNormalization(name='conv5_block1_2_bn')(x)
            x = Activation('relu',name='conv5_block1_2_relu')(x)
 
            x = Conv2D(2048, (1, 1), strides=(1, 1), padding='valid', name='conv5_block1_3_conv')(x)
            shortcut = Conv2D(2048, (1, 1), strides=(2, 2), padding='valid',name='conv5_block1_0_conv')(shortcut)
            x = BatchNormalization(name='conv5_block1_3_bn')(x)
            shortcut = BatchNormalization(name='conv5_block1_0_bn')(shortcut)          
 
            x = Add(name='conv5_block1_add')([x, shortcut])    
            x = Activation('relu',name='conv5_block1_out')(x)   
 
            shortcut = x               
        
        else:
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid',name='conv5_block' + str(i) + '_1_conv')(x)
            x = BatchNormalization(name='conv5_block' + str(i) + '_1_bn')(x)
            x = Activation('relu',name='conv5_block' + str(i) + '_1_relu')(x)
            
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same',name='conv5_block' + str(i) +'_2_conv')(x)
            x = BatchNormalization(name='conv5_block' + str(i) + '_2_bn')(x)
            x = Activation('relu',name='conv5_block' + str(i) + '_2_relu')(x)
 
            x = Conv2D(2048, (1, 1), strides=(1, 1), padding='valid',name='conv5_block' + str(i) + '_3_conv')(x)
            x = BatchNormalization(name='conv5_block' + str(i) + '_3_bn')(x)
            
            x = Add(name='conv5_block' + str(i) + '_add')([x, shortcut])
            x = Activation('relu',name='conv5_block' + str(i) + '_out')(x)
 
            shortcut = x                  
 
    return x
 
 
input_tensor = Input(shape=(224, 224, 3), dtype='float32', name='input')
x = conv1_layer(input_tensor)
x = conv2_layer(x)
x = conv3_layer(x)
x = conv4_layer(x)
x = conv5_layer(x)

#output_tensor = GlobalAveragePooling2D()(x)
x = GlobalAveragePooling2D()(x)
output_tensor = Dense(365, activation='softmax')(x)
 
resnet50 = Model(input_tensor, output_tensor)
resnet50.summary()


In [ ]:
import os
os.chdir('/content/drive/MyDrive/종합설계/myslowfast')
model = load_model(places365_path)
input = model.input
output = model.layers[-6].output
model = Model(input,output)
#model.summary()
#model.save_weights('/content/drive/MyDrive/종합설계/myslowfast/pre_trained/places365_exclude_top.h5')

In [ ]:
from model.model import SlowFast_body
slowfast = SlowFast_body((64,224,224,3),2)
slowfast.summary()

In [ ]:
#_layer.set_weights(np.array([layer_3d_weight,layer_3d_bias]))
#이런식으로 set
import numpy as np

layer_3d_weight = None
layer_3d_bias = None
weights_path = '/content/drive/MyDrive/종합설계/myslowfast/pre_trained/resnet50_places365.h5'
resnet50.load_weights(weights_path)
dic = {}
for layer in resnet50.layers:
  if layer.get_config()['name']=='conv4_block1_1_conv':
    print(layer.get_config()['name'], layer.weights[0].shape)
    dic[layer.get_config()['name']]=layer.weights[0]
    layer_3d_weight = np.repeat(layer.weights[0][np.newaxis,:,:,:,:], 3, axis=0)
    layer_3d_weight/=3

    layer_3d_bias = layer.weights[1]
    layer_3d_bias= layer_3d_bias.numpy()
    layer_3d_bias/=3
    
    
    


#print(type(layer_3d_weight))
#print(type(layer_3d_bias))
head_conv_three_list = []
for layer in slowfast.layers:
  config = layer.get_config()
  if config['name'].startswith('sequential'):
    for _layer in layer.layers:
      _config = _layer.get_config()
      if _config['name'].endswith('conv'):
        if _layer.get_weights()[0].shape[0]==3:
            #print(_layer.get_weights()[0].shape, _layer.get_weights()[1].shape)
            head_conv_three_list.append(_config['name'])

In [ ]:
cnt = 0
resnet50_conv_weights = []
resnet50_bn_weights = []
conv_weights = []
bn_weights = []
for layer in resnet50.layers:
  #print(layer.get_config()['name'])
  if layer.get_config()['name'].startswith('conv'):
    if layer.get_config()['name'].endswith('conv'):
      print(layer.get_config()['name'])
      #print(layer.get_weights()[0].shape)
      temp = []
      weights = layer.get_weights()
      for i in range(len(weights)):
        temp.append(weights[i])
      resnet50_conv_weights.append([layer.get_config()['name'], temp])
      cnt+=1
    elif layer.get_config()['name'].endswith('bn'):
      print(layer.get_config()['name'])
      #print(layer.get_weights()[1].shape)
      temp = []
      weights = layer.get_weights()
      for i in range(len(weights)):
        temp.append(weights[i])
      resnet50_bn_weights.append([layer.get_config()['name'], temp])
      cnt+=1
print(cnt)  
cnt=0

for layer in slowfast.layers:
  config = layer.get_config()
  if config['name'].startswith('sequential'):
    for _layer in layer.layers:
      _config = _layer.get_config()
      if _config['name'].startswith('slow'):
          if _config['name'].endswith('conv'):
            print(_config['name'])
            #print(_layer.get_weights()[0].shape)
            temp = []
            weights = _layer.get_weights()
            for i in range(len(weights)):
              temp.append(weights[i])
            conv_weights.append([_config['name'], temp])
            cnt+=1
          elif _config['name'].endswith('bn'):
            print(_config['name'])
            #print(_layer.get_weights()[0].shape)
            temp = []
            weights = _layer.get_weights()
            for i in range(len(weights)):
              temp.append(weights[i])
            bn_weights.append([_config['name'], temp])
            cnt+=1
  elif config['name'].startswith('slow'):
    if config['name'].endswith('conv'):
      print(config['name'])
      #print(layer.get_weights()[0].shape)
      temp = []
      weights = layer.get_weights()
      for i in range(len(weights)):
        temp.append(weights[i])
      conv_weights.append([layer.get_config()['name'], temp])
      cnt+=1
    elif config['name'].endswith('bn'):
      print(config['name'])
      #print(layer.get_weights()[0].shape)
      temp = []
      weights = layer.get_weights()
      for i in range(len(weights)):
        temp.append(weights[i])
      bn_weights.append([layer.get_config()['name'], temp])
      cnt+=1

print(cnt)


In [ ]:
'''
cnt=0
for i in range(len(conv_weights)):
  if conv_weights[i][1][0].shape != resnet50_conv_weights[i][1][0].shape:
    print(conv_weights[i][1][0].shape, resnet50_conv_weights[i][1][0].shape)
'''
'''
conv1_conv
conv1_bn
conv2_block1_1_conv
conv2_block1_1_bn
conv2_block1_2_conv
conv2_block1_2_bn
conv2_block1_3_conv
conv2_block1_0_conv
conv2_block1_3_bn
conv2_block1_0_bn


slow_res1_conv_conv
slow_res1_conv_bn
slow_res2_1_1_conv
slow_res2_1_1_bn
slow_res2_1_2_conv
slow_res2_1_2_bn
slow_res2_1_3_conv
slow_res2_0_3_bn
slow_res2_1_0_conv
'''
def name2name(resnetname):
  if resnetname=='conv1_conv':
    return 'slow_res1_conv_conv'
  elif resnetname=='conv1_bn':
    return 'slow_res1_conv_bn'
  else:
    convnumber,blocknumber,stage,operation = resnetname.split('_')
    #convnumber = resnetname.split('_')[0]
    #blocknumber = resnetname.split('_')[1]
    #stage = resnetname.split('_')[2]
    #operation = resnetname.split('_')[3]
    if operation=='conv':
      return 'slow_res{0}_{1}_{2}_conv'.format(convnumber[-1],blocknumber[-1],stage)
    elif operation=='bn':
      return 'slow_res{0}_{1}_{2}_bn'.format(convnumber[-1],blocknumber[-1],stage)


    layer_3d_bias = layer.weights[1]
    layer_3d_bias= layer_3d_bias.numpy()
    layer_3d_bias/=3
#conv 배열에 넣어주기
for i in range(len(resnet50_conv_weights)):
  current = resnet50_conv_weights[i]
  name = current[0]
  weights = current[1]
  name = name2name(name)
  
  for j in range(len(conv_weights)):
    if name == conv_weights[j][0]:
      if name in head_conv_three_list:
        weight = np.repeat(weights[0][np.newaxis,:,:,:,:], 3, axis=0)
        weight/=3
        bias = weights[1]
        conv_weights[j][1] = [weights,bias]
        break
      else:
        weight = np.repeat(weights[0][np.newaxis,:,:,:,:], 1, axis=0)
        bias = weights[1]
        conv_weights[j][1] = [weights,bias]
        break
        

#bn 배열에 넣어주기
for i in range(len(resnet50_bn_weights)):
  current = resnet50_bn_weights[i]
  name = current[0]
  weights = current[1]
  name = name2name(name)
  
  for j in range(len(bn_weights)):
    if name == bn_weights[j][0]:
        weight0 = weights[0]
        weight1 = weights[1]
        weight2 = weights[2]
        weight3 = weights[3]
        bn_weights[j][1] = [weight0,weight1,weight2,weight3]
        break
  
# conv_weights랑 bn_weights 가지고 알맞게 넣어주는데 채널수 팽창 필요

fast pathway만들기

In [ ]:
model = load_model(imagenet_path)
model.summary()

In [ ]:
os.chdir('/content/drive/MyDrive/종합설계/myslowfast')
from model.model import SlowFast_body
model = SlowFast_body(input_shape = (64,224,224,3),num_classes=2)

In [ ]:
model.summary()

In [ ]:
for layer in model.layers:
  print(len(layer.get_weights()))

In [ ]:
#name = 'conv' + str(stage) + '_' + 'block' + str(level) + 
for layer in model.layers:
  weights, bias = layer.get_weights()
  _weights = weights/3
  _bias = bias/3
  _weights for _ in range(3)

In [ ]:
for layer in model.layers:
  #print(layer.get_config()['name'])
  if name.endswith('conv'):
    #conv에 대해서
  elif name.endswith('bn'):
    #bn에 대해서